# Project - Team Isaiah Thomas 
## Kishan Rajasekhar and Jason Kandu

In [6]:
%pylab inline
plt.style.use('ggplot')
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

Populating the interactive namespace from numpy and matplotlib


In [7]:
# X_train = np.loadtxt('X_train.txt')
# Y_train = np.loadtxt('y_train.txt')
# X_test = np.loadtxt('X_test.txt')

In [8]:
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn import metrics
# ada_clf = AdaBoostClassifier()
# ada_clf.fit(X_train, Y_train)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)
# ada_predictions = ada_clf.predict_proba(X_val)
# # B = np.reshape(ada_predictions, [-1, 2])
# # print ada_predictions.shape
# # print ada_predictions
# np.savetxt('Yhat_adapredict.txt',np.vstack( (np.arange(len(ada_predictions)) , ada_predictions[:,1] )).T,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')
# # np.savetxt('Yhat_adapredict.txt',np.vstack( (np.arange(len(ada_predictions)) , ada_predictions[:] )).T,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')


In [9]:
# from sklearn.metrics import roc_curve, auc
# false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_val, ada_predictions[:,1])
# roc_auc = auc(false_positive_rate, true_positive_rate)
# print roc_auc

In [10]:
X = np.genfromtxt("data/X_train.txt",delimiter=' ')
Y = np.genfromtxt("data/Y_train.txt",delimiter=' ')
Xt,Xv,Yt,Yv = ml.splitData(X,Y,0.80)

Xe = np.genfromtxt('data/X_test.txt',delimiter=' ')

def auc(soft,Y):
    """Manual AUC function for applying to soft prediction vectors"""
    indices = np.argsort(soft)         # sort data by score value
    Y = Y[indices]
    sorted_soft = soft[indices]
    
    # compute rank (averaged for ties) of sorted data
    dif = np.hstack( ([True],np.diff(sorted_soft)!=0,[True]) )
    r1  = np.argwhere(dif).flatten()
    r2  = r1[0:-1] + 0.5*(r1[1:]-r1[0:-1]) + 0.5
    rnk = r2[np.cumsum(dif[:-1])-1]
    
    # number of true negatives and positives
    n0,n1 = sum(Y == 0), sum(Y == 1)
    
    # compute AUC using Mann-Whitney U statistic
    result = (np.sum(rnk[Y == 1]) - n1 * (n1 + 1.0) / 2.0) / n1 / n0
    return result


In [11]:

nClust = 15
np.random.seed(0)
Z,T,pZ,ll = ml.cluster.gmmEM(Xt[:10000,:],K=nClust,init='k++',max_iter=50)

Cluster_GMM = ml.bayes.gaussClassify()
# Manually copy the EM Gaussian components into the Gaussian Bayes Classifier:
Cluster_GMM.classes = np.arange(nClust)
Cluster_GMM.means = T['mu']
Cluster_GMM.covars = [ T['sig'][:,:,i]+.05*np.eye(Xt.shape[1]) for i in range(nClust) ]
Cluster_GMM.probs = T['pi']

# Find cluster membership probabilities for each data set:
XtC = Cluster_GMM.predictSoft(Xt)
XvC = Cluster_GMM.predictSoft(Xv)
XeC = Cluster_GMM.predictSoft(Xe)

# Create extended feature set:  features X times membership probability for each cluster
XtC2 = np.einsum('ij,ik->ijk',XtC,Xt).reshape((Xt.shape[0],Xt.shape[1]*nClust))
XvC2 = np.einsum('ij,ik->ijk',XvC,Xv).reshape((Xv.shape[0],Xv.shape[1]*nClust))
XeC2 = np.einsum('ij,ik->ijk',XeC,Xe).reshape((Xe.shape[0],Xe.shape[1]*nClust))

# Regress (should really use a classifier...)
linr2 = ml.linear.linearRegress(XtC2,Yt, reg=1e-3)
Pv3 = linr2.predict(XvC2)[:,0]
Pe3 = linr2.predict(XeC2)[:,0]

#toKaggle('Pe3.csv',Pe3)
print "3: Clustered LinRegress: MSE ~",linr2.mse(XvC2,Yv),'; AUC = ',auc(Pv3,Yv)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices